## 데이터 불러오기 및 전처리

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import tensorflow as tf
import numpy as np
import time
import pandas as pd
import os
import re

### 데이터 불러오기

In [ ]:
df = pd.read_csv('https://bit.ly/3n7iHQX')

In [ ]:
def clean_sentence(sentence):
    # 한글, 영어, 숫자를 제외한 문자 제거
    sentence = re.sub(r'[^0-9a-zA-Zㄱ-ㅎㅏ-ㅣ가-힣 ]',r'', sentence)
    # 문장의 끝을 표기
    sentence += ' #'
    return sentence

In [ ]:
df['text'] = df['text'].apply(clean_sentence)

In [ ]:
df.head()

,text
0,갤럭시S9 20만 원대 아이폰6S 0원 모비톡 가정의 달 이벤트갤럭시노트8 갤럭시S...
1,LG 그램 100만대 판매기념 한정판 나왔다LG전자가 그램 노트북 누적판매 100만...
2,이게 정말 LG폰이에요G7 씽큐 기분 좋은 스타트20일 서울 신촌역 앞 한 휴대폰 ...
3,애플 10억불vs 삼성 2800만불배상액 종지부 눈앞삼성애플 둥근모서리 디자인특허침...
4,삼성전자 5G 국제 표준 주도한다삼성전자가 5세대5G 이동통신 1차 표준 완성을 위...


### text 병합

In [ ]:
text = ' '.join(df['text'])

In [ ]:
# 총 문장의 길이
len(text)

222853

In [ ]:
print(text[:500]) #전처리 잘 되었는지 확인

갤럭시S9 20만 원대 아이폰6S 0원 모비톡 가정의 달 이벤트갤럭시노트8 갤럭시S9 갤럭시S8 갤럭시S7 갤럭시S7엣지 아이폰6S 아이폰X 아이폰8 G7 G6 V30 등 다양한 휴대폰 정보가 가득한 스마트폰 공동구매 및 거래 어플 모비톡의 가정의 달 이벤트가 화제다모비톡 단독으로 진행되는 5월 가정의 달 이벤트에 이용자들의 폭발적인 반응이 나타나고 있다 고가의 인기 스마트폰을 파격가에 판매한다는 사실에 각종 커뮤니티와 카페를 중심으로 화제를 모으고 있는 것 특히 갤럭시S9를 20만 원대 아이폰6S는 0원 할부원금을 앞세워 안드로이드와iOS인기 기종을 중심으로 큰 폭의 할인을 펼치는게 주된 요인으로 꼽힌다 모비톡 관계자에 따르면 고마운 사람들에게 감사한 마음을 담아 선물할 기회가 많은 5월 가정의 달을 맞아 공격적인 마케팅을 진행하고 있다며 독보적인 통신비 절약 어플로서 앞으로도 최선을 다하겠다고 밝혔다이 밖에도 모비톡은 갤럭시노트8 V30 구매 시 닌텐도 스위치를 증정한다 스마트폰


### text 전처리

In [ ]:
vocab = sorted(set(text)) #중복 글자 제거

In [ ]:
vocab[:20]

[' ',
 '#',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H']

In [ ]:
# 고유 글자의 숫자 확인
len(vocab)

1172

In [ ]:
vocab.append('?') #사용자의 입력이 없는 글자일때 ?로 표시하기 위해 추가

In [ ]:
#글자 -> index로 변환
char2idx = {u: i for i, u in enumerate(vocab)}

In [ ]:
#index -> 글자로 변환
idx2char = np.array(vocab)

### 단어 사전 만들기

In [ ]:
text_as_int = np.array([char2idx[c] for c in text])

In [ ]:
text_as_int

array([ 76, 394, 666, ..., 266,   0,   1])

In [ ]:
len(text_as_int)

222853

### 데이터셋 생성 및 EDA

In [ ]:
# 문장의 최대 길이를 지정
window_size = 100
shuffle_buffer = 1000
batch_size=128

In [ ]:
def windowed_dataset(series, window_size, shuffle_buffer, batch_size):
    series = tf.expand_dims(series, -1)
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(window_size + 1, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda x: x.batch(window_size + 1))
    ds = ds.shuffle(shuffle_buffer)
    ds = ds.map(lambda x: (x[:-1], x[1:]))
    return ds.batch(batch_size).prefetch(1).repeat()

In [ ]:
train_data = windowed_dataset(np.array(text_as_int), window_size, shuffle_buffer, batch_size)

In [ ]:
# 문자로 된 어휘 사전의 크기
vocab_size = len(vocab)
vocab_size

1173

### Sequential 모델 구현

In [ ]:
# 임베딩 차원
embedding_dim = 256

# RNN 유닛(unit) 개수
rnn_units = 1024

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=window_size),
    tf.keras.layers.LSTM(rnn_units,
                         return_sequences=True,
                         recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size, activation='softmax')
])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 256)          300288    
                                                                 
 lstm (LSTM)                 (None, 100, 1024)         5246976   
                                                                 
 dense (Dense)               (None, 100, 1173)         1202325   
                                                                 
Total params: 6,749,589
Trainable params: 6,749,589
Non-trainable params: 0
_________________________________________________________________


### Checkpoint 생성

In [ ]:
checkpoint_path = '/content/drive/MyDrive/Colab Notebooks/nlp/checkpoint.h5'

checkpointer = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    save_best_only=True,
    monitor='loss',
    verbose=1,
)

In [ ]:
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.2, patience=5, min_lr=0.0001)

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(), loss='sparse_categorical_crossentropy', metrics=['acc'])

In [ ]:
steps_per_epoch = (len(text_as_int) - window_size) // (batch_size)
steps_per_epoch

1740

In [ ]:
model.load_weights(checkpoint_path)

In [ ]:
model.fit(train_data,
          epochs=10,
          steps_per_epoch=steps_per_epoch,
          callbacks=[checkpointer, reduce_lr])

Epoch 1/10
1740/1740 [==============================] - ETA: 0s - loss: 0.2473 - acc: 0.9368
Epoch 1: loss improved from inf to 0.24726, saving model to /content/drive/MyDrive/Colab Notebooks/nlp/checkpoint.h5
1740/1740 [==============================] - 303s 169ms/step - loss: 0.2473 - acc: 0.9368 - lr: 0.0010
Epoch 2/10
1740/1740 [==============================] - ETA: 0s - loss: 0.2311 - acc: 0.9412
Epoch 2: loss improved from 0.24726 to 0.23112, saving model to /content/drive/MyDrive/Colab Notebooks/nlp/checkpoint.h5
1740/1740 [==============================] - 285s 164ms/step - loss: 0.2311 - acc: 0.9412 - lr: 0.0010
Epoch 3/10
1740/1740 [==============================] - ETA: 0s - loss: 0.1532 - acc: 0.9630
Epoch 3: loss improved from 0.23112 to 0.15320, saving model to /content/drive/MyDrive/Colab Notebooks/nlp/checkpoint.h5
1740/1740 [==============================] - 285s 164ms/step - loss: 0.1532 - acc: 0.9630 - lr: 0.0010
Epoch 4/10
1740/1740 [==============================]

### 뉴스기사 생성

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[1, None]),
    tf.keras.layers.LSTM(rnn_units,
                         return_sequences=True,
                         stateful=True,
                         recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
])

In [ ]:
model.load_weights(checkpoint_path)

In [ ]:
model.build(tf.TensorShape([1, None]))

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (1, None, 256)            300288    
                                                                 
 lstm_1 (LSTM)               (1, None, 1024)           5246976   
                                                                 
 dense_1 (Dense)             (1, None, 1173)           1202325   
                                                                 
Total params: 6,749,589
Trainable params: 6,749,589
Non-trainable params: 0
_________________________________________________________________


In [ ]:
def generate_text(model, start_string):
    # 평가 단계 (학습된 모델을 사용하여 텍스트 생성)

    # 생성할 문자의 수
    num_generate = 1000

    # 시작 문자열을 숫자로 변환(벡터화)
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # 결과 저장
    text_generated = []

    # 최적의 세팅을 찾기 위한 온도 설정
    temperature = 0.1

    # 여기에서 배치 크기 == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        # 배치 차원 제거
        predictions = tf.squeeze(predictions, 0)

        # 범주형 분포를 사용하여 모델에서 리턴한 단어 예측
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

        # 예측된 단어를 다음 입력으로 모델에 전달
        # 이전 은닉 상태와 함께
        input_eval = tf.expand_dims([predicted_id], 0)
        result_char = idx2char[predicted_id]

        # '#' 문자열을 만나면 종료
        if result_char == '#':
            break

        text_generated.append(result_char)

    return (start_string + ''.join(text_generated))

In [ ]:
print(generate_text(model, start_string=u"스마트폰 "))

스마트폰 제조사들은 자사의 플래그십 모델에 노치 디자인을 잇따라 채택하고 있다 올 초 스페인에서 열린 모바일월드콩그레스MWC2018에서 에이서스 오포 비보 화웨이 등이 노치를 채택한 스마트폰을 공개했다LG전자 역시 G7에 노치 디자인을 도입했다CPR은 노치 디자인은 화면의 크기를 극대화할 수 있다는 점과 혁신의 부족을 만회할 수 있는 카드라고 분석했다다만CPR은 애플의 노치와 그외의 노치는 기능적으로 다름을 분명히 했다CPR은 애플은 홈버튼과 지문센서를 없애고 페이스ID라는 신기술을 탑재하기 위해 노치를 선택했다면서 그런데 다른 제조사들은 노치를 도입했으면서도 그 기술적 이유를 알 수가 없다고 말했다이어 일부 제조사의 경우 소프트웨어의 측면에서도 기능적으로도 무의미한 따라하기이지만 그들에게는 애플과 유사한 디자인을 채택할 필요가 있었다고 분석했다타룬 파닥TarunPathakCPR애널리스트는 안드로이드 진영이 대거 애플 따라하기에 나서면서 최소한 스마트폰 시장에서 2년 정도는 노치 트렌드가 계속될 것이라고 예상했다안드로이드 운영체제 개발사인 구글도 노치 디자인을 플랫폼 차원에서 지원한다 구글은 지난달 차세대 운영체제 안드로이드P 개발자 프리뷰에서 노치 디자인을 지원하는 디스플레이 컷아웃DisplayCutout 기능을 선보였다 


In [ ]:
print(generate_text(model, start_string=u"삼성 "))

삼성 제품에 판매할 수 있는 것이다 게다가 로봇 바리스타의 가격도 사람 바리스타 1명의 1년 연봉 수준인 약 2만5000달러약 2700만원정도다미국 유명 엔젤투자자 제이슨 칼라캐니스는 사람 바리스타는 커피 주문을 잘못 받거나 음료 맛을 제각각으로 만들고 가계통신비를 인하하자는 정책이기 때문에 1분기에 적자로 전환됐고 올해와 내년에도 계속 실적 저조 가능성이 있는 것인 만큼 네이버 최고경영자가 적극 나서야 할 것이라고 주장했다한성숙 네이버 대표는 9일 기자간담회를 열어 추가 댓글대책을 발표한다 
